In [1]:
from ufl import *
import numpy as np
from mpi4py import MPI


def printer(elements):
    if isinstance( elements,list):
        for elem in elements:
            print('{:}   :::   {:}'.format(elem,type(elem)))
    else: print('{:} : {:}'.format(elements,type(elements)))


def ti(tit=''):
    if tit == 'end': print('{:-^80}'.format('End block'))
    else: print('{:*^80}'.format(tit))

In [2]:
cell = Cell(cellname='triangle', geometric_dimension=2)

el = FiniteElement(
    family='CG',
    cell=cell,
    degree=1,
    )

i = Index()
x, y = SpatialCoordinate(cell)
arg = Argument(number=1, function_space=el)
coef = Coefficient(count=1, function_space=el)

ti('Preprare')
printer([arg, coef, x])

a = sin(x + 2)
a2 = a * dx
a3 = a * ds
v = as_vector([sin(y) + 2, x])
v2 = v**2 * dx
trial = TrialFunction(el)
test = TestFunction(el)
const = Constant(count=1, domain=cell)
ti('Derivatives')
printer([a, a2, v2, trial, test, const, v[i] * v[i]])
ti('end')

ti('Variables')
g = coef * 2
g2 = variable(g)
h = g2**2
printer([g, g2, h, diff(h, g2), diff(h, coef)])

ti('Replace')
coef1, coef2, coef3 = Coefficient(el), Coefficient(el), Coefficient(el)
a_coef = coef1 * coef2**coef3
printer(a_coef)
printer(replace(a_coef, {coef1: 5, coef2: 2, coef3: 3}))

ti('Operands')
printer(*a.ufl_operands)
b = sin(cos(x + y*2)) + 2*a
printer(b.ufl_operands)
ti('end')

print(b.ufl_free_indices)
print(b.ufl_shape)
print(v.ufl_shape)
v3 = v**2
print(v3.ufl_shape)

ti('coefs')
test1 = (coef1 * coef2**coef3 + coef1*coef3*x)
test2 = replace(test1, {coef1: 5, coef2: 2})
test3 = replace(test2, {coef3: 2, coef2: 2})
printer([test1, test2, test3])
ti('end')

************************************Preprare************************************
v_1   :::   <class 'ufl.argument.Argument'>
w_1   :::   <class 'ufl.coefficient.Coefficient'>
x[0]   :::   <class 'ufl.indexed.Indexed'>
**********************************Derivatives***********************************
sin(2 + x[0])   :::   <class 'ufl.mathfunctions.Sin'>
{ sin(2 + x[0]) } * dx(<Mesh #-1>[everywhere], {})   :::   <class 'ufl.form.Form'>
{ ([2 + sin(x[1]), x[0]]) : ([2 + sin(x[1]), x[0]]) } * dx(<Mesh #-1>[everywhere], {})   :::   <class 'ufl.form.Form'>
v_1   :::   <class 'ufl.argument.Argument'>
v_0   :::   <class 'ufl.argument.Argument'>
c_1   :::   <class 'ufl.constant.Constant'>
sum_{i_8} ([2 + sin(x[1]), x[0]])[i_8] * ([2 + sin(x[1]), x[0]])[i_8]    :::   <class 'ufl.indexsum.IndexSum'>
-----------------------------------End block------------------------------------
***********************************Variables************************************
2 * w_1   :::   <class 'ufl.algebra.Prod

In [3]:
element2 = VectorElement(
    family='CG',
    cell=cell,
    degree=1,
    )

domain2 = AbstractDomain(geometric_dimension=2, topological_dimension=2)
space2 = FunctionSpace(domain=domain2, element=element2)
coef2 = Coefficient(count=1, function_space=space2)
printer(coef2)

Expected non-abstract domain for initalization of function space.


UFLException: Expected non-abstract domain for initalization of function space.

# to fem

In [4]:
from dolfinx import fem,mesh

In [5]:
domain = mesh.create_unit_square(
    comm=MPI.COMM_WORLD,
    cell_type=mesh.CellType.triangle,
    nx=10,
    ny=10,
    )

space = fem.FunctionSpace(mesh=domain, element=el)
coef0_f = fem.Function(space)

In [ ]:
coef0_f.

In [29]:
t01 = coef0_f.ufl_domain()
t02 = t01.ufl_cargo()
t02

In [25]:
t1 = coef.ufl_domain()
t2 = t1.ufl_cargo()
t3 = coef.ufl_function_space()
t4 = coef.ufl_function_space()
t1

Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), -1)

In [24]:
coef_exp = fem.Expression(
    ufl_expression=coef,
    X=space.element.interpolation_points(),
    )


AttributeError: 'NoneType' object has no attribute 'comm'